Fake New Detection

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
import string
import re

Data collection

In [2]:
# read data 
data_fake = pd.read_csv('Fake.csv')
data_true = pd.read_csv('True.csv')

# Display first 10 rows of each dataset
print("Fake News Data:\n", data_fake.head(10))
print("\nTrue News Data:\n", data_true.head(10))

Fake News Data:
                                                title  \
0   Donald Trump Sends Out Embarrassing New Year’...   
1   Drunk Bragging Trump Staffer Started Russian ...   
2   Sheriff David Clarke Becomes An Internet Joke...   
3   Trump Is So Obsessed He Even Has Obama’s Name...   
4   Pope Francis Just Called Out Donald Trump Dur...   
5   Racist Alabama Cops Brutalize Black Boy While...   
6   Fresh Off The Golf Course, Trump Lashes Out A...   
7   Trump Said Some INSANELY Racist Stuff Inside ...   
8   Former CIA Director Slams Trump Over UN Bully...   
9   WATCH: Brand-New Pro-Trump Ad Features So Muc...   

                                                text subject  \
0  Donald Trump just couldn t wish all Americans ...    News   
1  House Intelligence Committee Chairman Devin Nu...    News   
2  On Friday, it was revealed that former Milwauk...    News   
3  On Christmas day, Donald Trump announced that ...    News   
4  Pope Francis used his annual Christmas Day 

In [3]:
data_fake["class"] = 0
data_true["class"] = 1

In [4]:
data_fake.shape,data_true.shape

((23481, 5), (21417, 5))

In [7]:
# Drop only existing indices in data_fake
data_fake_testing = data_fake.tail(10)
for i in range(23481, 23470, -1):
    if i in data_fake.index:
        data_fake.drop(i, axis=0, inplace=True)

# Drop only existing indices in data_true
data_true_testing = data_true.tail(10)
for i in range(21417, 21406, -1):
    if i in data_true.index:
        data_true.drop(i, axis=0, inplace=True)

In [8]:
data_testing = pd.concat([data_fake_testing, data_true_testing], axis = 0)
data_testing.to_csv("testing.csv")

In [9]:
#combine the data
dataFT_marge = pd.concat([data_fake, data_true],axis = 0)
print(dataFT_marge.head(10))
print(dataFT_marge.info(10))

                                               title  \
0   Donald Trump Sends Out Embarrassing New Year’...   
1   Drunk Bragging Trump Staffer Started Russian ...   
2   Sheriff David Clarke Becomes An Internet Joke...   
3   Trump Is So Obsessed He Even Has Obama’s Name...   
4   Pope Francis Just Called Out Donald Trump Dur...   
5   Racist Alabama Cops Brutalize Black Boy While...   
6   Fresh Off The Golf Course, Trump Lashes Out A...   
7   Trump Said Some INSANELY Racist Stuff Inside ...   
8   Former CIA Director Slams Trump Over UN Bully...   
9   WATCH: Brand-New Pro-Trump Ad Features So Muc...   

                                                text subject  \
0  Donald Trump just couldn t wish all Americans ...    News   
1  House Intelligence Committee Chairman Devin Nu...    News   
2  On Friday, it was revealed that former Milwauk...    News   
3  On Christmas day, Donald Trump announced that ...    News   
4  Pope Francis used his annual Christmas Day mes...    News   

In [10]:
data = dataFT_marge.drop(["title","subject","date"],axis= 1)
print(data.head(10))

                                                text  class
0  Donald Trump just couldn t wish all Americans ...      0
1  House Intelligence Committee Chairman Devin Nu...      0
2  On Friday, it was revealed that former Milwauk...      0
3  On Christmas day, Donald Trump announced that ...      0
4  Pope Francis used his annual Christmas Day mes...      0
5  The number of cases of cops brutalizing and ki...      0
6  Donald Trump spent a good portion of his day a...      0
7  In the wake of yet another court decision that...      0
8  Many people have raised the alarm regarding th...      0
9  Just when you might have thought we d get a br...      0


In [11]:
#shuffle the dataset 
data = data.sample(frac = 1)
print(data.head(10))

                                                    text  class
7322   Coolest. President. Ever.During his scheduled ...      0
18449  Rep. Debbie Wasserman Schultz dropped a big bo...      0
10321  Where s George Soros when his top  fact  check...      0
7413   WASHINGTON (Reuters) - U.S. House of Represent...      1
8151   NEW YORK (Reuters) - Nearly half of American a...      1
12383  The press was not happy and took to Twitter to...      0
13749  THE ARTICLE BELOW IS FROM POLITICO   This arti...      0
8364   Not a day goes by when a prominent figure on t...      0
9982   WASHINGTON (Reuters) - Chuck Grassley and Merr...      1
167    Despite what many racists say and regardless o...      0


In [12]:
# counting the number of missing values in the dataset
data.isnull().sum()

text     0
class    0
dtype: int64

In [13]:
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer

# Load English stopwords
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [14]:
# Download necessary NLTK data files
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
    
# Initialize Stemming and Lemmatization
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

# Load English stopwords
stop_words = set(stopwords.words('english'))

def word_drop(text):

    # Convert to lowercase
    text = text.lower()

    # Remove punctuation and special characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Remove text inside square brackets
    text = re.sub(r'\[.*?\]', '', text)

    # Remove URLs
    text = re.sub(r'https?://\S+|www\.\S+', '', text)

    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Remove numbers
    text = re.sub(r'\d+', '', text)

    # Tokenize the text into words
    words = text.split()

    # Remove stopwords
    words = [word for word in words if word not in stop_words]

    # Apply Stemming
    words = [stemmer.stem(word) for word in words]

    # Apply Lemmatization
    words = [lemmatizer.lemmatize(word) for word in words]

    # Join words back into a string
    return " ".join(words)

# Apply text preprocessing function to the "text" column
data["text"] = data["text"].apply(word_drop)

# Print the first 10 rows and the shape of the DataFrame
print(data.head(10))
print("Shape of data:", data.shape)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


                                                    text  class
7322   coolest presid everdur schedul stop bueno air ...      0
18449  rep debbi wasserman schultz drop big bomb inte...      0
10321  georg soro top fact checker aka conserv news s...      0
7413   washington reuter u hous repres speaker paul r...      1
8151   new york reuter nearli half american adult con...      1
12383  press happi took twitter temper tantrumth way ...      0
13749  articl politico articl perfect exampl main str...      0
8364   day goe promin figur right remind u good guy g...      0
9982   washington reuter chuck grassley merrick garla...      1
167    despit mani racist say regardless handpick sta...      0
Shape of data: (44878, 2)


In [15]:
#dependentand independent variables
x = data["text"]
y = data["class"]

In [16]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.25)

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
xv_train = vectorizer.fit_transform(x_train)  
xv_test =  vectorizer.transform(x_test)

In [18]:
from sklearn.ensemble import RandomForestClassifier
RF = RandomForestClassifier(random_state = 42)
RF.fit(xv_train, y_train)

RandomForestClassifier(random_state=42)

In [19]:
RF.score(xv_test, y_test)

0.9867201426024955

In [20]:
prediction_RF = RF.predict(xv_test)

In [21]:
print(classification_report(y_test, prediction_RF))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5825
           1       0.98      0.99      0.99      5395

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



In [22]:
# Save the trained model and vectorizer
import pickle

with open("rf_model.pkl", "wb") as model_file:
    pickle.dump(RF, model_file)

with open("vectorizer.pkl", "wb") as vectorizer_file:
    pickle.dump(vectorizer, vectorizer_file)

print("Model and vectorizer saved successfully!")

Model and vectorizer saved successfully!


In [23]:
def output_label(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Not A Fake News"
        
def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(word_drop)

    new_x_test = new_def_test["text"]
    new_xv_test = vectorizer.transform(new_x_test)
    prediction_RF = RF.predict(new_xv_test)
    return print("\n\nRF Prediction:{}".format(output_label(prediction_RF))) 

In [24]:
news = str(input())
manual_testing(news)

 KILLED: Rancher and protest spokesman Robert  Lavoy  Finicum (Image Source: RT International)21st Century Wire says New eye witness testimony indicates that the FBI and other federal agencies were never fired upon, but have shot and killed leading protester, Robert  Lavoy  Finicum, and wounding one other in a elaborate operation that took place along a rural highway near Burns, Oregon.For 26 days since the protest and occupation began at the Malheur Wildlife Refuge outside of Burns, Oregon there were no one was hurt and there were no injuries. That all changed early Tuesday evening when the FBI and Oregon State Police raised the ante, bringing the 4 week standoff to a dramatic ending.As detailed in yesterday s report, according to 18 yr old passenger and eyewitness, Victoria Sharp, dozens of federal vehicles and armed agents had set-up a pre-positioned roadblocks along an isolated stretch of rural Highway 395 before intercepting two vehicles and firing upon one of them, killing ranche



RF Prediction:Fake News


In [25]:
!streamlit run app.py

^C
